In [1]:
import torch
from concrete.ml.torch.compile import compile_brevitas_qat_model
from mingpt.model import GPT
from mingpt.utils import CfgNode as CN

/Users/davidhu/Documents/code/ml/65610_project/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/davidhu/Documents/code/ml/65610_project/venv/lib/python3.10/site-packages/threadpoolctl.py:1223: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
from fhegpt.qmodel import QuantCasualSelfAttention

In [3]:
config = CN(
    n_embd = 768,
    n_layer = 12,
    n_head=12,
    attn_pdrop=0.1,
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    block_size=1000
)
qcsa = QuantCasualSelfAttention(config)

In [4]:
print(qcsa)

QuantCasualSelfAttention(
  (quant_inp): QuantIdentity(
    (input_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
    )
    (act_quant): ActQuantProxyFromInjector(
      (_zero_hw_sentinel): StatelessBuffer()
      (fused_activation_quant_proxy): FusedActivationQuantProxy(
        (activation_impl): Identity()
        (tensor_quant): RescalingIntQuant(
          (int_quant): IntQuant(
            (float_to_int_impl): RoundSte()
            (tensor_clamp_impl): TensorClamp()
            (delay_wrapper): DelayWrapper(
              (delay_impl): _NoDelay()
            )
          )
          (scaling_impl): ParameterFromRuntimeStatsScaling(
            (stats_input_view_shape_impl): OverTensorView()
            (stats): _Stats(
              (stats_impl): AbsPercentile()
            )
            (restrict_scaling): _RestrictValue(
              (restrict_value_impl): FloatRestrictValue()
            )
            (clamp_scaling): _ClampValue(
           

In [5]:
torch_input = torch.randn(1, 10, config.n_embd)
quantized_model = compile_brevitas_qat_model(qcsa, torch_input)

/Mul and {'/MatMul_output_0'} and {1: <concrete.ml.quantization.quantizers.QuantizedArray object at 0x158375ea0>} and <concrete.ml.quantization.quantizers.QuantizationOptions object at 0x158376110> and {}
/Div and {'/ReduceSum_output_0', '/Reshape_5_output_0'} and {} and <concrete.ml.quantization.quantizers.QuantizationOptions object at 0x1583768f0> and {}


AssertionError: Do not support this type of operation between encrypted tensors

# 